In [1]:
# Check GPU availability and CUDA version
!nvidia-smi
!nvcc --version

# Import necessary modules
import os

# Set the current working directory
HOME = os.getcwd()
print("HOME:", HOME)

Wed Aug 20 13:15:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   53C    P8             13W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Installing SAM2 and optuna

In [3]:
# # Clone the SAM 2 repository
!git clone https://github.com/facebookresearch/segment-anything-2.git

# Change to the repository directory
%cd segment-anything-2

# Install the SAM 2 package
!pip install -e .

# Install additional packages
!pip install supervision jupyter_bbox_widget

!pip install optuna

Cloning into 'segment-anything-2'...
remote: Enumerating objects: 1070, done.
remote: Total 1070 (delta 0), reused 0 (delta 0), pack-reused 1070 (from 1)
Receiving objects: 100% (1070/1070), 128.11 MiB | 23.77 MiB/s, done.
Resolving deltas: 100% (381/381), done.
/content/segment-anything-2
Obtaining file:///content/segment-anything-2
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 8.1 MB/s eta 0:00:00
  Building editable for SAM-2 (pyproject.toml) ... done
  Created wheel for SAM-2: filename=sam_2-1.0-0.editable-cp312-cp312-linux_x86_64.whl size=13864 sha256=8d2eabd4d56cb8896450dd4fd29cc99bf51a838d8deac225d2ee437b237a8db9
  Stored in directory: /tmp/pi

In [4]:
# # Create a directory for checkpoints
!mkdir -p checkpoints

# Download the model checkpoints
!wget -q https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_tiny.pt -P checkpoints
!wget -q https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_small.pt -P checkpoints
!wget -q https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_base_plus.pt -P checkpoints
!wget -q https://dl.fbaipublicfiles.com/segment_anything_2/072824/sam2_hiera_large.pt -P checkpoints

## Input batches

In [5]:
a_input_folders = ['/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_input/1. PSD95/1. Original File Names/',
                '/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_input/2. SY38/1. Original File Names/',
               '/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_input/3. MXO4/1. Original File Names/',
               '/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_input/4. GFAP/1. Original File Names/'
               ]


a_gt_folders = [
            '/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_output/1. PSD95/2. Decrypted File Names/',
            '/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_output/2. SY38/2. Decrypted File Names/',
            '/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_output/3. MXO4/2. Decrypted File Names/',
            '/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_output/4. GFAP/2. Decrypted File Names/']

b_input_folders = ['/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_input/1. PSD95/1. Original File Names/',
                '/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_input/2. SY38/1. Original File Names/',
               '/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_input/3. MXO4/1. Original File Names/',
               '/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_input/4. GFAP/1. Original File Names/'
               ]


b_gt_folders = [
            '/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_output/1. PSD95/2. Decrypted File Names/',
            '/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_output/2. SY38/2. Decrypted File Names/',
            '/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_output/3. MXO4/2. Decrypted File Names/',
            '/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_output/4. GFAP/2. Decrypted File Names/']

## Pre-trained Models

In [15]:
psd_model_paths = ['/content/drive/MyDrive/Colab Notebooks/models/SAM2/PSD_train_test_same_batch_2001_tune_2000.torch',
'/content/drive/MyDrive/Colab Notebooks/models/SAM2/PSD_train_test_diff_batch_2001_tune_2000.torch',
'/content/drive/MyDrive/Colab Notebooks/models/SAM2/PSD_train_test_mixed_batch_2001_tune_2000.torch']


sy38_model_paths = ['/content/drive/MyDrive/Colab Notebooks/models/SAM2/SY38_train_test_same_batch_2001_tune_2000.torch',
'/content/drive/MyDrive/Colab Notebooks/models/SAM2/SY38_train_test_diff_batch_2001_tune_2000.torch',
'/content/drive/MyDrive/Colab Notebooks/models/SAM2/SY38_train_test_mixed_batch_2001_tune_2000.torch']


mxo4_model_paths = ['/content/drive/MyDrive/Colab Notebooks/models/SAM2/MXO4_train_test_same_batch_2000_tune_2000.torch',
'/content/drive/MyDrive/Colab Notebooks/models/SAM2/MXO4_train_test_diff_batch_2000_tune_2000.torch',
'/content/drive/MyDrive/Colab Notebooks/models/SAM2/MXO4_train_test_mixed_batch_2000_tune_2000.torch']


gfap_model_paths = ['/content/drive/MyDrive/Colab Notebooks/models/SAM2/GFAP_train_test_same_batch_2001_tune_2000.torch',
'/content/drive/MyDrive/Colab Notebooks/models/SAM2/GFAP_train_test_diff_batch_2001_tune_2000.torch',
'/content/drive/MyDrive/Colab Notebooks/models/SAM2/GFAP_train_test_mixed_batch_2001_tune_2000.torch']

## Re-tune and train example

In [8]:
command = f'python "/content/drive/MyDrive/Colab Notebooks/Segmentation_Pipeline/sam_retraining.py" -mn PSD -tts 0.8 -mode train_test_same_batch \
 -input1 "{b_input_folders[0]}" -gt1 "{b_gt_folders[0]}" -input2 "{a_input_folders[0]}" -gt2 "{a_gt_folders[0]}" \
 -ts 0.3 -tunep 0.4 -trainp 0.5 -testp 0.6 -numt 2 -nume 500 -run_mode tune -model_path "{psd_model_paths[0]}"'

!{command}

Namespace(model_name='PSD', train_test_split=0.8, train_test_mode='train_test_same_batch', input_folders_1='/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_input/1. PSD95/1. Original File Names/', gt_folders_1='/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_output/1. PSD95/2. Decrypted File Names/', input_folders_2='/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_input/1. PSD95/1. Original File Names/', gt_folders_2='/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_output/1. PSD95/2. Decrypted File Names/', tune_split=0.3, tune_indice_prop=0.4, train_indice_prop=0.5, test_indice_prop=0.6, num_tune_trials=2, num_epoch=500, run_mode='tune', pretrained_model_path='/content/drive/MyDrive/Colab Notebooks/models/SAM2/PSD_train_test_same_batch_2001_tune')
Create train and test data by reading and formatting input files and applying train-test split

/content/drive/MyDrive/Colab Notebooks/Lab_imag

## Load retrained model example

In [14]:
command = f'python "/content/drive/MyDrive/Colab Notebooks/Segmentation_Pipeline/sam_retraining.py" -mn PSD -tts 0.8 -mode train_test_same_batch \
 -input1 "{b_input_folders[0]}" -gt1 "{b_gt_folders[0]}" -input2 "{a_input_folders[0]}" -gt2 "{a_gt_folders[0]}" \
 -ts 0.3 -tunep 1 -trainp 1 -testp 1 -numt 5 -nume 99 -run_mode eval -model_path "{psd_model_paths[0]}"'

!{command}

Namespace(model_name='PSD', train_test_split=0.8, train_test_mode='train_test_same_batch', input_folders_1='/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_input/1. PSD95/1. Original File Names/', gt_folders_1='/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_output/1. PSD95/2. Decrypted File Names/', input_folders_2='/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_input/1. PSD95/1. Original File Names/', gt_folders_2='/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_output/1. PSD95/2. Decrypted File Names/', tune_split=0.3, tune_indice_prop=1.0, train_indice_prop=1.0, test_indice_prop=1.0, num_tune_trials=5, num_epoch=99, run_mode='eval', pretrained_model_path='/content/drive/MyDrive/Colab Notebooks/models/SAM2/PSD_train_test_same_batch_2001_tune_2000.torch')
Create train and test data by reading and formatting input files and applying train-test split

/content/drive/MyDrive/Colab Notebook